# <center> The Horsehead nebula dataset </center>

## <center> Analyzing portions of the field of view </center>

In [ ]:
# %% load modules
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '.', '..'))

import numpy as np 

# TOAST toolboxs
from toolboxs.toolbox_physics.toolbox_physics import compute_moments
from toolboxs.toolbox_plot.toolbox_plot import plot_maps, show_LoS

# to show image results
from IPython.display import Image
from IPython.display import IFrame

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors


In [ ]:
# paths
path_dataset = '***/TOAST/examples/data-horsehead/inputs'

To illustrate how derive map of estimations with `TOAST`, this notebook is focusing one a dense region ($A_v > 16 \text{ mag}$) within the Horsehead. 
The corresponding field of view map `Fov_dense_region.py` is provided in `examples/data-horsehead/inputs`. Please, <font color='red'> rename it `FoV.py`</font>.

In [ ]:
FoV = np.load(f'{path_dataset}/FoV.npy', allow_pickle=True) # position-position-velocity cubes

# check on the FoV map 
name_fig = 'FoV'
fig = plot_maps(
    maps=[FoV],
    titles=[r'Studied field of view'],
    array_coordinates=True, 
    cmap = ['gray']
    )

## The sandwich model

To describe region with dense cores, we model the sight-line as a sandwich (see the figure below). The corresponding filled template `model_analyze_map.py` is provided in the folder `TOAST/`. <font color='red'> Rename it `model.py`</font> (in `model.py`, modify the section `paths` as well). 

**Note**: see details about the sandwich model in reference [1].

In [ ]:
# sketch of the model 
# theta is the vector of parameters characterizing the gas (Tkin, nH2, colden, FWMH, C_V)
img = Image(f'schema/sandwich.png', width=600)
img

`TOAST` performs multiprocessing. To indeed processing pixels in parallel, in `model.py`, section `verbose and plot setting`, put `PARALLELISM, POLL_SIZE = True, number-of-cores`, where `POLL_SIZE` is the number of cores to use among the ones of your machine. Otherwise, set `PARALLELISM = False, POLL_SIZE = 0`.

In [ ]:
# launch the model fitting 

os.system('python .././launch_model_fitting.py')

**Note**: one can check the progression of the process by looking at the file `results_rw`.

In [ ]:
# checking the estimation results, from the random walk (the search in the gridded space), 
# written in readable files `result_rw`

# NLL is the Negative Log-Likelihood, the criterion that we are minimizing
# Walkers are the vectors of parameters spread in the space. 
# The number of walkers found at the minimum NLL can be used as an insight of the convergence.

file = open(f'analyze_map/outputs/results_rw', 'r')
textfile = file.read()
print(textfile)

**Notes**: As seen in the notebook `analyse_one_pixel.ipynb`, one can refine those estimations with the gradient descent by relaunching the model fitting after setting in `model.py`, section `model fitting settings`, `optimization='gd'`.

### Result visualization 

In [ ]:
# load results 
# vector theta : {log(Tkin), log(nH2), log(N(mol_ref)), FWHM, CV}, for each layer
maps_theta_rw = np.load(f'analyze_map/outputs/estimations/maps_theta_rw.npy', allow_pickle = True)

number_of_params = maps_theta_rw.shape[2]
number_of_layers = maps_theta_rw.shape[3]

# show maps of Tkin, nH2, N(mol_ref), FWHM, C_V

folder = 'maps_estimations'
os.system(f'mkdir analyze_map/{folder}')

titles = [
    r'log($T_{\mathrm{kin}}$ /K)',
    r'log($n_{\mathrm{H}_2}$ /cm$^{-3}$)',
    r'log($N$ /cm$^{-2}$)',
    r'FWHM [km/s]',
    r'$C_V$ [km/s]'
]

# for colormap 
FPS = np.load(f'data-horsehead/grids-Tex-tau/radexUnifSphere/FPS.npy', allow_pickle = True)
vmin_vmax = [
            [10**np.min(FPS[0]), 10**np.max(FPS[0])], 
            [10**np.min(FPS[1]), 10**np.max(FPS[1])],
            [10**np.min(FPS[2]), 10**np.max(FPS[2])],
            [np.min(FPS[3]), np.max(FPS[3])],
            [9.5, 11.5]]
format_colorbar=['%.f', '', '', '%.1f', '%.1f']
minorticks_on = [True, True, True, False, False]
cmap = ['jet', 'jet', 'jet', 'seismic', 'seismic']
nicknames_layers = ['foreground', 'inner', 'background']
norm = [
    colors.LogNorm(vmin = vmin_vmax[0][0], vmax = vmin_vmax[0][1]), 
    colors.LogNorm(vmin = vmin_vmax[1][0], vmax = vmin_vmax[1][1]), 
    colors.LogNorm(vmin = vmin_vmax[2][0], vmax = vmin_vmax[2][1]), 
    colors.Normalize(vmin = vmin_vmax[3][0], vmax = vmin_vmax[3][1]),
    colors.Normalize(vmin=9.5, vmax=11.5)
]

nrows, ncols = number_of_params, number_of_layers
fig, axs = plt.subplots(nrows=nrows, ncols=ncols,
                                figsize=(10, 10), layout="constrained")
for i in range(nrows):

    for j in range(ncols):
        ax = axs[i, j]

        if j == 0 : 
            ax.set_ylabel(f'{titles[i]}')
        if i == 0 : 
            ax.set_title(f'Layer {j+1} ({nicknames_layers[j]})')

        map = maps_theta_rw[:, :, i, j]

        if i in [0, 1, 2] : # log(Tkin), log(nH2), log(N)
            map = 10 ** map

        img = ax.imshow(
                map,
                origin='lower',
                cmap = cmap[i],
                norm = norm[i]
                  )
        
        if format_colorbar[i] == '' : 
            colorBar = plt.colorbar(
                    img, 
                    ax = ax)
        else:
            colorBar = plt.colorbar(
                img, 
                ax = ax, 
                format = format_colorbar[i]
                )
        if minorticks_on[i] != False : 
            colorBar.ax.minorticks_on()
   
fig.savefig(f'analyze_map/{folder}/maps_theta_rw.pdf')


In [ ]:
# show maps of column density 
# column densities of all species 
maps_log10_N_rw = np.load(f'analyze_map/outputs/estimations/maps_log10_N_rw.npy', allow_pickle = True)

number_of_species = maps_log10_N_rw.shape[2]
number_of_layers = maps_log10_N_rw.shape[3]

titles = [
    '12co',
    '13co',
    'c18o',
    'hcop',
    'h13cop'
]

# for colormap 
FPS = np.load(f'data-horsehead/grids-Tex-tau/radexUnifSphere/FPS.npy', allow_pickle = True)
colden_ratios = np.load(f'data-horsehead/inputs/colden_ratios.npy', allow_pickle = True)

vmin_mol_ref, vmax_mol_ref = np.min(FPS[2]), np.max(FPS[2]) # log10(N(13co))
vmin_vmax = [
            [10**(vmin_mol_ref + colden_ratios.item().get('12co')), 10**(vmax_mol_ref + colden_ratios.item().get('12co'))], 
            [10**(vmin_mol_ref + colden_ratios.item().get('13co')), 10**(vmax_mol_ref + colden_ratios.item().get('13co'))], 
            [10**(vmin_mol_ref + colden_ratios.item().get('c18o')), 10**(vmax_mol_ref + colden_ratios.item().get('c18o'))], 
            [10**(vmin_mol_ref + colden_ratios.item().get('hcop')), 10**(vmax_mol_ref + colden_ratios.item().get('hcop'))], 
            [10**(vmin_mol_ref + colden_ratios.item().get('h13cop')), 10**(vmax_mol_ref + colden_ratios.item().get('h13cop'))]
            ]
cmap = 'jet'
nicknames_layers = ['foreground', 'inner', 'background']
norm = [
    colors.LogNorm(vmin = vmin_vmax[0][0], vmax = vmin_vmax[0][1]), 
    colors.LogNorm(vmin = vmin_vmax[1][0], vmax = vmin_vmax[1][1]), 
    colors.LogNorm(vmin = vmin_vmax[2][0], vmax = vmin_vmax[2][1]), 
    colors.LogNorm(vmin = vmin_vmax[3][0], vmax = vmin_vmax[3][1]),
    colors.LogNorm(vmin = vmin_vmax[4][0], vmax = vmin_vmax[4][1])
]

nrows, ncols = number_of_params, number_of_layers
fig, axs = plt.subplots(nrows=nrows, ncols=ncols,
                                figsize=(10, 10), layout="constrained")
for i in range(nrows):

    for j in range(ncols):
        ax = axs[i, j]

        if j == 0 : 
            ax.set_ylabel(f'{titles[i]}')
        if i == 0 : 
            ax.set_title(f'Layer {j+1} ({nicknames_layers[j]})')

        map = maps_log10_N_rw[:, :, i, j]
        map = 10 ** map
        map = np.where(np.isnan(FoV), np.nan, map)

        img = ax.imshow(
                map,
                origin='lower',
                cmap = cmap,
                #norm = norm[i]
                  )
        
        colorBar = plt.colorbar(
                    img, 
                    ax = ax
                    )
        colorBar.ax.minorticks_on()
   
fig.savefig(f'analyze_map/{folder}/maps_theta_rw.pdf')

